In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 
import seaborn as sns
from matplotlib import pyplot as plt

from IPython.core.display import display, HTML
from PIL import Image
from io import BytesIO
import base64

import os
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Input, BatchNormalization, ReLU, AveragePooling2D
from tensorflow.keras.layers import Conv2D, Flatten, Dropout, UpSampling2D
from tensorflow.keras.layers import Lambda, Subtract, Add
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import np_utils

%matplotlib inline

In [ ]:
# set variables 
main_folder = '../input/celeba-dataset/'
images_folder = main_folder + 'img_align_celeba/img_align_celeba/'
EXAMPLE_PIC = images_folder + '000506.jpg'

In [ ]:
# import the data set that include the attribute for each picture
df_attr = pd.read_csv(main_folder + 'list_attr_celeba.csv')
df_attr.set_index('image_id', inplace=True)
df_attr.replace(to_replace=-1, value=0, inplace=True) #replace -1 by 0
df_attr.shape

In [ ]:
glass = df_attr["Eyeglasses"]
# In gender array 0-no glass while 1-glass
glass.head(5)

In [ ]:
plt.title('yes or no glasses')
sns.countplot(y='Eyeglasses', data=df_attr, color="c")
plt.show()

In [ ]:
df_partition = pd.read_csv(main_folder + 'list_eval_partition.csv')
# df_partition.head()

In [ ]:
df_partition['partition'].value_counts().sort_index()

In [ ]:
df_partition.set_index('image_id', inplace=True)
df_par_attr = df_partition.join(glass, how='inner')
df_par_attr.head()

In [ ]:
def load_reshape_img(fname):
    x = cv2.imread(fname)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (64,64)).astype('float32') / 255.
    x = x.reshape((1,) + x.shape)

    return x


def generate_df(partition, attr, num_samples):
    '''
    partition
        0 -> train
        1 -> validation
        2 -> test
    
    '''
    
    df_ = df_par_attr[(df_par_attr['partition'] == partition) 
                           & (df_par_attr[attr] == 0)].sample(int(num_samples/2))
    df_ = pd.concat([df_,
                      df_par_attr[(df_par_attr['partition'] == partition) 
                                  & (df_par_attr[attr] == 1)].sample(int(num_samples/2))])

    # for Train and Validation
    if partition != 2:
        x_ = np.array([load_reshape_img(images_folder + fname) for fname in df_.index])
        x_ = x_.reshape(x_.shape[0], 64, 64, 3)
        y_ = np_utils.to_categorical(df_[attr],2)
    # for Test
    else:
        x_ = []
        y_ = []

        for index, target in df_.iterrows():
            im = cv2.imread(images_folder + index)
            im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (IMG_WIDTH, IMG_HEIGHT)).astype(np.float32) / 255.0
            im = np.expand_dims(im, axis =0)
            x_.append(im)
            y_.append(target[attr])

    return x_, y_

In [ ]:
TRAINING_SAMPLES = 10000
VALIDATION_SAMPLES = 2000
IMG_WIDTH = 64
IMG_HEIGHT = 64

x_train, y_train = generate_df(0, 'Eyeglasses', TRAINING_SAMPLES)
x_valid, y_valid = generate_df(1, 'Eyeglasses', VALIDATION_SAMPLES)

In [ ]:
count = 0
for i in range(10000):
    if (y_train[i]==np.array([1,0])).all():
        count += 1
print(x_train.shape)
count

In [ ]:
input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)
kernel_size = 5
gamma_init = tf.random_normal_initializer(1., 0.02)

inputs = Input(shape=input_shape, name='classifier_input')
x = Conv2D(filters=64, kernel_size=kernel_size, strides=2, padding='same')(inputs)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2D(filters=128, kernel_size=kernel_size, strides=2, padding='same')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Conv2D(filters=256, kernel_size=kernel_size, strides=2, padding='same')(x)
x = BatchNormalization(gamma_initializer=gamma_init, trainable=True)(x)
x = ReLU()(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(units=1, activation = 'sigmoid')(x)

classifier = Model(inputs, x, name='classifier')
# classifier.summary()

In [ ]:
classifier = load_model('../input/glass-classifier/glass_cf_15_sigmoid.tf')

In [ ]:
classifier.compile(loss='bce', optimizer='adam')

In [ ]:
batch_size = 128

# SIGMOID
y_train_sigmoid = np.array([np.argmax(one_hot)for one_hot in y_train])
y_valid_sigmoid = np.array([np.argmax(one_hot)for one_hot in y_valid])


train_datagen =  ImageDataGenerator()
train_datagen.fit(x_train)
train_generator = train_datagen.flow(
x_train, y_train_sigmoid,
batch_size=batch_size,
)

In [ ]:
hist = classifier.fit_generator(train_generator
                     , validation_data = (x_valid, y_valid_sigmoid)
                      , steps_per_epoch= TRAINING_SAMPLES/batch_size
                      , epochs= 15
                    )

In [ ]:
plt.figure(figsize=(18, 4))
plt.plot(hist.history['loss'], label = 'train')
plt.plot(hist.history['val_loss'], label = 'valid')
plt.legend()
plt.title('Loss Function')
plt.savefig('loss.pdf')
plt.show()

In [ ]:
test_img = load_reshape_img(images_folder+'200041.jpg')
plt.imshow(test_img[0])

In [ ]:
print(classifier(test_img))

In [ ]:
classifier.save("glass_cf_15_sigmoid.tf")